In [1]:
pip install pandas openai scikit-learn bert-score sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7a14b77dea2a14f5a6b4d00dba9eef8561ab246e84aa678523e8587d37ae99f2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import pandas as pd
import openai
import os
from sklearn.model_selection import train_test_split
from typing import List, Dict, Tuple, Any
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer

# Configure your API key
openai.api_key = openai_secret  # Make sure to set this environment variable

# Read the files
with open('questions.txt', 'r', encoding='utf-8') as qf:
    questions = [line.strip() for line in qf.readlines()]

with open('Jagadeesh_Answers.txt', 'r', encoding='utf-8') as tf:
    my_answers = [line.strip() for line in tf.readlines()]

with open('Sandeep_Answers.txt', 'r', encoding='utf-8') as mf:
    teammate_answers = [line.strip() for line in mf.readlines()]

if not (len(questions) == len(teammate_answers) == len(my_answers)):
    raise ValueError("Mismatch in number of questions and answers.")

df = pd.DataFrame({
    'Question': questions,
    'Teammate_Answer': teammate_answers,
    'My_Answer': my_answers
})

In [4]:
# Show the first few rows
df.head(5)

,Question,Teammate_Answer,My_Answer
0,Describe how you make your favorite breakfast.,I generally make my breakfast pretty quick unl...,My favourite breakfast is always having bread ...
1,Walk me through your typical weekday routine.,"I generally wake up at around 8, have my routi...",I try to wake up early. And I will get up earl...
2,How do you usually prepare for a big event or ...,I like to keep myself calm before big events o...,Check at what time the event is at. If it’s a ...
3,Explain how you’d help someone set up a new ph...,"I will first check if it IOS or Android, then ...","I only setup someone else’s phone, only if the..."
4,What do you do when you’re running late?,"Ahh… First, I will try not to be late. If by a...",If it’s some important event or meeting. And I...


In [5]:
train_df, test_df = train_test_split(df, test_size=10, random_state=42)
few_shot_examples = train_df[['Question', 'Teammate_Answer']].rename(columns={'Teammate_Answer': 'Answer'}).to_dict(orient='records')

In [6]:
train_qs = set(train_df['Question'])
test_qs = set(test_df['Question'])

if train_qs.isdisjoint(test_qs):
    print("✅ Train and test sets are disjoint. No data leakage.")
else:
    print("❌ Overlap detected between train and test sets!")
    print(train_qs.intersection(test_qs))

✅ Train and test sets are disjoint. No data leakage.


## Zero Shot

In [7]:
def create_style_description(style_attributes: Dict[str, Any]) -> str:
    """
    Create a detailed description of the answering style based on provided attributes.
    Returns: A string description of the style
    """
    description = "When answering questions, use the following style:\n\n"

    # Add each style attribute to the description
    for attribute, value in style_attributes.items():
        if attribute == "examples" and isinstance(value, list):
            description += f"- {attribute.capitalize()}: Use phrasing similar to these examples: {', '.join(value)}\n"
        elif attribute == "additional" and isinstance(value, list):
            for item in value:
                description += f"- {item}\n"
        else:
            description += f"- {attribute.capitalize()}: {value}\n"

    return description


def get_styled_answer_zero_shot(question: str, style_attributes: Dict[str, Any], model: str = "gpt-4") -> str:
    """
    Get an answer to a question using a specified style without examples.
    Returns: Generated answer in the specified style
    """
    style_description = create_style_description(style_attributes)

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": f"You are an assistant that answers questions in a specific style. {style_description}"},
            {"role": "user", "content": question}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

# - tone: formal, casual, technical, etc.
# - length: short, medium, long
# - format: paragraph, bullet points, numbered list, etc.
# - vocabulary: simple, advanced, technical, etc.
# - perspective: first person, third person, etc.
# - additional: any other style attributes

# Define the style you want to copy
style_to_copy = {
    "tone": "friendly talk, normal conversation",
    "length": "concise, typically 2-3 sentences",
    "format": "start with a direct answer, then provide brief supporting details, if you are exceeding the recommended length",
    "vocabulary": "sinmple and casual but accessible, avoiding overly technical jargon",
    "perspective": "use second person ('you') when appropriate to engage the reader",
    "structure": "begin with the core answer, then explanation, then practical application if relevant",
    "additional": [
        "Always include one concrete example when explaining abstract concepts",
        # "End with a hint of broader implications or next steps when appropriate",
        "Can use hedging language like 'perhaps' or 'maybe' - be definitive"
    ]
}

# Question to answer in the specified style
# question = "What causes thunderstorms?"
# question = "Describe how you make your favorite breakfast?"
question = "What do you do when you’re running late?"

# Get the styled answer
styled_answer = get_styled_answer_zero_shot(question, style_to_copy)
print(f"Question: {question}")
print(f"Answer: {styled_answer}")

Question: What do you do when you’re running late?
Answer: When you realize you're running late, you should immediately let the person waiting for you know about it. Send them a quick message or give them a call to inform them of your delay, so they're not left in the dark. It's a good idea to provide an estimated time of arrival, if possible. For instance, you could text your friend saying, "Hey, I'm running a bit late, caught in traffic. Should be there by 3:15 pm. Sorry for the inconvenience!"


## Few Shot

In [8]:
def create_few_shot_prompt(examples: List[Dict[str, str]], new_question: str) -> str:
    prompt = "Answer the following questions in the style demonstrated by these examples:\n\n"

    # Add examples
    for i, example in enumerate(examples):
        prompt += f"Question {i+1}: {example['Question']}\n"
        prompt += f"Answer {i+1}: {example['Answer']}\n\n"

    # Add the new question
    prompt += f"Question: {new_question}\nAnswer:"
    return prompt

def get_styled_answer_few_shot(examples: List[Dict[str, str]], new_question: str, model: str = "gpt-4o") -> str:

    prompt = create_few_shot_prompt(examples, new_question)
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an assistant that answers questions in the exact style of the examples provided."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()


# Sample question-answer pairs
examples = [
    {
        "Question": "Describe how you make your favorite breakfast?",
        "Answer": "My favourite breakfast is always having bread with eggs, as it is easier to make. Just break down 2 eggs, add some salt and pepper to it, stir and put it in a microwave. While that’s happening, I also put 2 bread slices in the toaster. After both are done, add some ranch & chipotle ranch and sandwich the eggs b/w the bread. We are done :)"
    },
    {
        "Question": "Walk me through your typical weekday routine.",
        "Answer": "I try to wake up early. And I will get up early, only if I don’t hit the snooze like 10 times. If hungry, eat breakfast, while going through the notifications, while simultaneously catching up all the new AI stuff. Go to class, if I have one that day. Else try some new code or model that I find online, most though my youtube recommendations."
    },
    {
        "Question": "How do you usually prepare for a big event or meeting?",
        "Answer": "Check at what time the event is at. If it’s a casual event, no preparation is needed. Else, if it’s something like an interview or career fair, make sure to put that on my calenders and put some remainders for that event date and time, as well as the previous day. So that i don’t forget. Go through the prep and rehearse in my mind of how the event would be."
    }
]

# New question to answer in the same style
# new_question = "What causes thunderstorms?"
new_question = "What do you do when you’re running late?"

# Get the styled answer
styled_answer = get_styled_answer_few_shot(examples, new_question)
print(f"Question: {new_question}")
print(f"Answer: {styled_answer}")

Question: What do you do when you’re running late?
Answer: First, I panic a little because that’s just my natural response. Then I quickly calculate how much time I actually have left and prioritize what absolutely needs to be done. If I’m still home, I’ll skip some steps like making the bed or doing dishes. For getting ready, I go for the essentials: brush teeth, throw on whatever clothes are clean and decent, and rush out the door. On the way, I text or call whoever I’m meeting to let them know I’m running late. It’s not ideal, but it happens.


# Fine-tuning

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from time import perf_counter
import torch

# Load the model and tokenizer from HuggingFace
model_name = "Jgdshkovi/tinyllama-finetuned-for-p1-v0"
print(f"Loading model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use half precision to reduce memory usage
    device_map="auto"  # Automatically determine the best device mapping
)

Loading model: Jgdshkovi/tinyllama-finetuned-for-p1-v0


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
def formatted_prompt(question) -> str:
    return f"<|user|>\n{question}</s>\n<|assistant|>"

def generate_response(user_input):
    prompt = formatted_prompt(user_input)

    # Create inputs
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # Setup generation config
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        max_new_tokens=120,
        pad_token_id=tokenizer.eos_token_id
    )

    # Time the inference
    start_time = perf_counter()

    # Generate response
    outputs = model.generate(**inputs, generation_config=generation_config)

    # Decode and print the response
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's part
    if "<|assistant|>" in decoded:
      response = decoded.split("<|assistant|>")[-1].strip()
    else:
      response = decoded.strip()

    # Calculate and print inference time
    output_time = perf_counter() - start_time

    return response

In [11]:
sample_question = "What's something random you remember from school days?"
print(f"Sample Question: {sample_question}")
generate_response(sample_question)

Sample Question: What's something random you remember from school days?


'Remember the time when we used to play cricket on the grass, with the elimination rule. It still remains one of my favorite games. We use to make a big hill on the grass, by piling up the debris. Don’t go down the hill if it is raining. » That was our Elders’ work. Now, lower the uphill and make the hill by piling up the debris. » Up they come! Today, everyone will bow to that hill. Don’t go down the hill if it is raining'

In [12]:
# ====================
# Evaluation
# ====================

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# --- ROUGE-L ---
def compute_rouge_l(preds, refs):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, pred)['rougeL'].fmeasure for pred, ref in zip(preds, refs)]
    return sum(scores) / len(scores)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
print("\n=======================")
print("🧪 Running Tests on Shared Question Set...\n")

impersonation_refs = []
generalization_refs = []
few_preds = []
zero_preds = []
local_preds = []

for i, row in test_df.iterrows():
    q = row['Question']
    teammate_ans = row['Teammate_Answer']
    own_ans = row['My_Answer']

    pred_few = get_styled_answer_few_shot(few_shot_examples, q)
    pred_zero = get_styled_answer_zero_shot(q, style_to_copy)
    pred_local = generate_response(q)

    impersonation_refs.append(teammate_ans)
    generalization_refs.append(own_ans)
    few_preds.append(pred_few)
    zero_preds.append(pred_zero)
    local_preds.append(pred_local)

    print(f"\nQ: {q}")
    print(f"👤 Teammate: {teammate_ans}")
    print(f"🧑‍🎓 You: {own_ans}")
    print(f"🤖 Few-Shot: {pred_few}")
    print(f"🤖 Zero-Shot: {pred_zero}")
    print(f"🤖 Fine-tuned Model: {pred_local}")
    print("-" * 60)


🧪 Running Tests on Shared Question Set...


Q: Describe how you make your favorite breakfast.
👤 Teammate: I generally make my breakfast pretty quick unless I have anything special in my mind on that day, especially weekends. I would opt for Oats, or anything quick like upma. For Oats, I would soak oats with milk, yogurt, honey and add chocolate powder to add some flavour, overnight.
🧑‍🎓 You: My favourite breakfast is always having bread with eggs, as it is easier to make. Just break down 2 eggs, add some salt and pepper to it, stir and put it in a microwave. While that’s happening, I also put 2 bread slices in the toaster. After both are done, add some ranch & chipotle ranch and sandwich the eggs b/w the bread. We are done :)
🤖 Few-Shot: I start by making a classic South Indian breakfast — dosa. First, I soak rice and urad dal overnight. Then, I blend them into a smooth batter in the morning, adding a bit of salt. I heat a non-stick pan, pour a ladle of batter, and spread it evenly in

In [14]:
# ====================
# Evaluation: Impersonation
# ====================
print("\n=======================")
print("📘 Impersonation Performance (vs. Teammate Style)")
_, _, few_f1_imp = bert_score(few_preds, impersonation_refs, lang="en", rescale_with_baseline=True)
_, _, zero_f1_imp = bert_score(zero_preds, impersonation_refs, lang="en", rescale_with_baseline=True)
_, _, local_f1_imp = bert_score(local_preds, impersonation_refs, lang="en", rescale_with_baseline=True)

imp_embed_ref = embedding_model.encode(impersonation_refs, convert_to_tensor=True)
few_embed = embedding_model.encode(few_preds, convert_to_tensor=True)
zero_embed = embedding_model.encode(zero_preds, convert_to_tensor=True)
local_embed = embedding_model.encode(local_preds, convert_to_tensor=True)

few_cos_sim_imp = util.cos_sim(imp_embed_ref, few_embed).diag()
zero_cos_sim_imp = util.cos_sim(imp_embed_ref, zero_embed).diag()
local_cos_sim_imp = util.cos_sim(imp_embed_ref, local_embed).diag()

rouge_imp_few = compute_rouge_l(few_preds, impersonation_refs)
rouge_imp_zero = compute_rouge_l(zero_preds, impersonation_refs)
rouge_imp_local = compute_rouge_l(local_preds, impersonation_refs)

print("-"*50)
print(f"✅ Fine-tuned BERTScore F1:    {local_f1_imp.mean().item():.4f}")
print(f"✅ Fine-tuned Style Similarity:{local_cos_sim_imp.mean().item():.4f}")
print(f"✅ Fine-tuned ROUGE-L:          {rouge_imp_local:.4f}")
print("-"*50)
print(f"✅ Few-Shot BERTScore F1:     {few_f1_imp.mean().item():.4f}")
print(f"✅ Few-Shot Style Similarity: {few_cos_sim_imp.mean().item():.4f}")
print(f"✅ Few-Shot ROUGE-L:           {rouge_imp_few:.4f}")
print("-"*50)
print(f"✅ Zero-Shot BERTScore F1:    {zero_f1_imp.mean().item():.4f}")
print(f"✅ Zero-Shot Style Similarity:{zero_cos_sim_imp.mean().item():.4f}")
print(f"✅ Zero-Shot ROUGE-L:          {rouge_imp_zero:.4f}")


📘 Impersonation Performance (vs. Teammate Style)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--------------------------------------------------
✅ Fine-tuned BERTScore F1:    0.0588
✅ Fine-tuned Style Similarity:0.4753
✅ Fine-tuned ROUGE-L:          0.1273
--------------------------------------------------
✅ Few-Shot BERTScore F1:     0.1824
✅ Few-Shot Style Similarity: 0.5366
✅ Few-Shot ROUGE-L:           0.1922
--------------------------------------------------
✅ Zero-Shot BERTScore F1:    0.0932
✅ Zero-Shot Style Similarity:0.4893
✅ Zero-Shot ROUGE-L:          0.1460


In [15]:
# ====================
# Evaluation: Generalizability
# ====================
print("\n=======================")
print("🌐 Generalizability Performance (vs. My Style)")
_, _, few_f1_gen = bert_score(few_preds, generalization_refs, lang="en", rescale_with_baseline=True)
_, _, zero_f1_gen = bert_score(zero_preds, generalization_refs, lang="en", rescale_with_baseline=True)
_, _, local_f1_gen = bert_score(local_preds, generalization_refs, lang="en", rescale_with_baseline=True)

gen_embed_ref = embedding_model.encode(generalization_refs, convert_to_tensor=True)
few_embed_gen = embedding_model.encode(few_preds, convert_to_tensor=True)
zero_embed_gen = embedding_model.encode(zero_preds, convert_to_tensor=True)
local_embed_gen = embedding_model.encode(local_preds, convert_to_tensor=True)

few_cos_sim_gen = util.cos_sim(gen_embed_ref, few_embed_gen).diag()
zero_cos_sim_gen = util.cos_sim(gen_embed_ref, zero_embed_gen).diag()
local_cos_sim_gen = util.cos_sim(gen_embed_ref, local_embed_gen).diag()

rouge_gen_few = compute_rouge_l(few_preds, generalization_refs)
rouge_gen_zero = compute_rouge_l(zero_preds, generalization_refs)
rouge_gen_local = compute_rouge_l(local_preds, generalization_refs)

print("-"*50)
print(f"✅ Fine-tuned BERTScore F1:    {local_f1_gen.mean().item():.4f}")
print(f"✅ Fine-tuned Style Similarity:{local_cos_sim_gen.mean().item():.4f}")
print(f"✅ Fine-tuned ROUGE-L:          {rouge_gen_local:.4f}")
print("-"*50)
print(f"✅ Few-Shot BERTScore F1:     {few_f1_gen.mean().item():.4f}")
print(f"✅ Few-Shot Style Similarity: {few_cos_sim_gen.mean().item():.4f}")
print(f"✅ Few-Shot ROUGE-L:           {rouge_gen_few:.4f}")
print("-"*50)
print(f"✅ Zero-Shot BERTScore F1:    {zero_f1_gen.mean().item():.4f}")
print(f"✅ Zero-Shot Style Similarity:{zero_cos_sim_gen.mean().item():.4f}")
print(f"✅ Zero-Shot ROUGE-L:          {rouge_gen_zero:.4f}")


🌐 Generalizability Performance (vs. My Style)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--------------------------------------------------
✅ Fine-tuned BERTScore F1:    0.0884
✅ Fine-tuned Style Similarity:0.5023
✅ Fine-tuned ROUGE-L:          0.1415
--------------------------------------------------
✅ Few-Shot BERTScore F1:     0.0953
✅ Few-Shot Style Similarity: 0.3926
✅ Few-Shot ROUGE-L:           0.1434
--------------------------------------------------
✅ Zero-Shot BERTScore F1:    0.0572
✅ Zero-Shot Style Similarity:0.4238
✅ Zero-Shot ROUGE-L:          0.1212
